In [1]:
# Imports:
import numpy as np
import pandas as pd
import math
import seaborn as sns
import matplotlib.pyplot as plt
import pickle
import sklearn
from sklearn.metrics import mean_squared_error
import plotly.express as px
from scipy import stats
import torch
from torch.utils.data import Dataset, TensorDataset
from torch.utils.data import DataLoader

In [2]:
#set seed for reproducibility
random_state = 42
np.random.seed(random_state)

--------

### Horizon Analyses:

In [3]:
# import horizon files 
with open('/Users/mvilenko/Library/CloudStorage/OneDrive-PayPal/CPI_HRNN - version 2.0/mayas_project/basic_model_canada/data/hor1_test_data_dict.pickle', 'rb') as f:
    canada_hor_1_test_dict = pickle.load(f)

with open('/Users/mvilenko/Library/CloudStorage/OneDrive-PayPal/CPI_HRNN - version 2.0/mayas_project/basic_model_canada/data/hor2_test_data_dict.pickle', 'rb') as f:
    canada_hor_2_test_dict = pickle.load(f)

with open('/Users/mvilenko/Library/CloudStorage/OneDrive-PayPal/CPI_HRNN - version 2.0/mayas_project/basic_model_canada/data/hor3_test_data_dict.pickle', 'rb') as f:
    canada_hor_3_test_dict = pickle.load(f)

with open('/Users/mvilenko/Library/CloudStorage/OneDrive-PayPal/CPI_HRNN - version 2.0/mayas_project/basic_model_canada/data/hor4_test_data_dict.pickle', 'rb') as f:
    canada_hor_4_test_dict = pickle.load(f)

with open('/Users/mvilenko/Library/CloudStorage/OneDrive-PayPal/CPI_HRNN - version 2.0/mayas_project/basic_model_canada/data/hor8_test_data_dict.pickle', 'rb') as f:
    canada_hor_8_test_dict = pickle.load(f)

In [4]:
canada_hor_1_test_dict['All-items'].shape

(26, 14)

In [5]:
import sys        
sys.path.append('/Users/mvilenko/Library/CloudStorage/OneDrive-PayPal/CPI_HRNN - version 2.0/mayas_project/basic_model_canada')  

from GRU_model import *
from config import * 

Lr = 0.09782661222201432


In [6]:
def load_checkpoint(checkpoint_path, model, optimizer):
    """
    checkpoint_path: path to save checkpoint
    model: model that we want to load checkpoint parameters into       
    optimizer: optimizer we defined in previous training
    """
    # load check point
    checkpoint = torch.load(checkpoint_path)
    # initialize state_dict from checkpoint to model
    model.load_state_dict(checkpoint['state_dict'])
    # initialize optimizer from checkpoint to optimizer
    optimizer.load_state_dict(checkpoint['optimizer'])
    # initialize valid_loss_min from checkpoint to valid_loss_min
    valid_loss_min = checkpoint['valid_loss_min']
    # return model, optimizer, epoch value, min validation loss 
    return model, optimizer, checkpoint['epoch'], valid_loss_min

In [7]:
Device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_forecast_steps = 8

category_horizon_dict = {}

for category_name in list(canada_hor_1_test_dict.keys()):
    # load model
    cat_model = GRUModel(input_dim=Features, hidden_dim=HiddenSize, layer_dim=LayersDim, output_dim=OutputDim, dropout_prob=DropoutProb)
    cat_optimizer = torch.optim.AdamW(cat_model.parameters(), lr=Lr)
    cat_model.to(Device)

    try:
        category_model, optimizer, checkpoint, valid_loss_min = load_checkpoint('/Users/mvilenko/Library/CloudStorage/OneDrive-PayPal/CPI_HRNN - version 2.0/mayas_project/basic_model_canada/checkpoints/best_checkpoints/'+category_name+'.pt', cat_model, cat_optimizer)

    except:
        print(f'{category_name} not found')

    horizon_dict = {}

    # Generate forecasts step by step
    for step in range(num_forecast_steps):
        with torch.no_grad():
            # create dataloader
            x_test = canada_hor_1_test_dict[category_name].iloc[:,:-1].to_numpy()
            y_test= canada_hor_1_test_dict[category_name].iloc[:,-1:].to_numpy()

            x_test = torch.from_numpy(x_test).to(torch.float32)
            y_test = torch.from_numpy(y_test).to(torch.float32)

            test_dataset = TensorDataset(x_test, y_test)

            test_dataloader =  DataLoader(test_dataset, batch_size=BatchSize, shuffle=False)

            for inputs, labels in test_dataloader:
                # since each batch contains 32 samples, and the largest df is of size 32, then we dont need to concat the outputs per batch per category.
                inputs = inputs.view(inputs.shape[0], SequenceLength, Features)
                #print(f'inputs shape: {inputs.shape}')
                #print(f'labels shape: {labels.shape}')
                inputs, labels = inputs.to(Device), labels.to(Device)
                out = category_model(inputs)
                out_df = pd.DataFrame(out)
                
                first = canada_hor_1_test_dict[category_name].iloc[:,1:].reset_index(drop = True).copy()
                print(f'first shape: {first.shape}')
                second = out_df.reset_index(drop = True).rename(columns = {0: 'Inflation t+'+str(2+step)})
                print(f'second shape: {second.shape}')

                canada_hor_1_test_dict[category_name] = pd.concat([first, second],axis=1) #prediction

                horizon_dict[step+2] = canada_hor_1_test_dict[category_name]

    category_horizon_dict[category_name] = horizon_dict
    


first shape: (26, 13)
second shape: (26, 1)
first shape: (26, 13)
second shape: (26, 1)
first shape: (26, 13)
second shape: (26, 1)
first shape: (26, 13)
second shape: (26, 1)
first shape: (26, 13)
second shape: (26, 1)
first shape: (26, 13)
second shape: (26, 1)
first shape: (26, 13)
second shape: (26, 1)
first shape: (26, 13)
second shape: (26, 1)
first shape: (26, 13)
second shape: (26, 1)
first shape: (26, 13)
second shape: (26, 1)
first shape: (26, 13)
second shape: (26, 1)
first shape: (26, 13)
second shape: (26, 1)
first shape: (26, 13)
second shape: (26, 1)
first shape: (26, 13)
second shape: (26, 1)
first shape: (26, 13)
second shape: (26, 1)
first shape: (26, 13)
second shape: (26, 1)
first shape: (26, 13)
second shape: (26, 1)
first shape: (26, 13)
second shape: (26, 1)
first shape: (26, 13)
second shape: (26, 1)
first shape: (26, 13)
second shape: (26, 1)
first shape: (26, 13)
second shape: (26, 1)
first shape: (26, 13)
second shape: (26, 1)
first shape: (26, 13)
second sha

In [8]:
def avg_rmse_hor(prediction_dict, actual_dict, horizon):
    mse_lst = []
    for key in list(prediction_dict.keys()):
        if prediction_dict[key] == {}:
            continue
        predictions = prediction_dict[key][horizon].iloc[:,-1:].reset_index(drop = True)
        actuals = actual_dict[key].iloc[:,-1:].reset_index(drop = True)

        y_pred = predictions.values
        y_actual = actuals.values

        non_null_indices = ~np.isnan(y_pred) & ~np.isnan(y_actual)

        predicted_values_non_null = y_pred[non_null_indices]
        observed_values_non_null = y_actual[non_null_indices]

        #print(f'y pred: {predicted_values_non_null}')
        #print(f'y actual: {observed_values_non_null}')
        mse = mean_squared_error(predicted_values_non_null, observed_values_non_null)
        mse_lst.append(mse)
    
    rmse_list = list(map(np.sqrt,mse_lst))
    avg_rmse = np.mean(rmse_list)
    rmse_std = np.std(rmse_list)
    
    print(f'RMSE:  {avg_rmse}')
    print(f'MSE std:  {rmse_std}')
    print(f'interval: {[avg_rmse-rmse_std, avg_rmse+rmse_std]}')

    return avg_rmse,rmse_std

In [9]:
avg_rmse_hor(category_horizon_dict, canada_hor_2_test_dict, 2)

RMSE:  2.304352147764165
MSE std:  2.174141056881341
interval: [0.130211090882824, 4.478493204645506]


(2.304352147764165, 2.174141056881341)

In [10]:
avg_rmse_hor(category_horizon_dict, canada_hor_3_test_dict, 3)

RMSE:  2.371379077747592
MSE std:  2.2884010167149267
interval: [0.08297806103266536, 4.659780094462519]


(2.371379077747592, 2.2884010167149267)

In [11]:
avg_rmse_hor(category_horizon_dict, canada_hor_4_test_dict, 4)

RMSE:  2.3898104758804877
MSE std:  2.3165502781181235
interval: [0.07326019776236414, 4.706360753998611]


(2.3898104758804877, 2.3165502781181235)

In [12]:
avg_rmse_hor(category_horizon_dict, canada_hor_8_test_dict, 8)

RMSE:  2.463601066805257
MSE std:  2.476582272601532
interval: [-0.012981205796275042, 4.940183339406789]


(2.463601066805257, 2.476582272601532)

In [13]:
#category_horizon_dict - prediction dict per category per horizon
# us_hor_x_test_dict - actual dict per category for horizon x

In [14]:
with open('/Users/mvilenko/Library/CloudStorage/OneDrive-PayPal/CPI_HRNN - version 2.0/pickle files/bi_directional_canada_2_period_dataset_dict.pickle', 'rb') as f:
    hor_2_raw_dataset_dict = pickle.load(f)

with open('/Users/mvilenko/Library/CloudStorage/OneDrive-PayPal/CPI_HRNN - version 2.0/pickle files/bi_directional_canada_3_period_dataset_dict.pickle', 'rb') as f:
    hor_3_raw_dataset_dict = pickle.load(f)

with open('/Users/mvilenko/Library/CloudStorage/OneDrive-PayPal/CPI_HRNN - version 2.0/pickle files/bi_directional_canada_4_period_dataset_dict.pickle', 'rb') as f:
    hor_4_raw_dataset_dict = pickle.load(f)

with open('/Users/mvilenko/Library/CloudStorage/OneDrive-PayPal/CPI_HRNN - version 2.0/pickle files/bi_directional_canada_8_period_dataset_dict.pickle', 'rb') as f:
    hor_8_raw_dataset_dict = pickle.load(f)

In [15]:
def create_test_dataframe(raw_dataset_dict: dict, horizon:int):
    test_dict = {}
    for key in raw_dataset_dict.keys():
        df = raw_dataset_dict[key][['Category', 'Date', 'Year', 'Inflation t+'+str(horizon)]]
        df.dropna(inplace=True)
        df.rename(columns={'Inflation t+'+str(horizon): 'Actual Horizon '+str(horizon)}, inplace=True)
        target_df = df[df['Year'] > Year]
        target_df = target_df.reset_index(drop = True)
        test_dict[key] = target_df
    return test_dict

In [16]:
test_dict_hor_2 = create_test_dataframe(hor_2_raw_dataset_dict, 2)
test_dict_hor_3 = create_test_dataframe(hor_3_raw_dataset_dict, 3)
test_dict_hor_4 = create_test_dataframe(hor_4_raw_dataset_dict, 4)
test_dict_hor_8 = create_test_dataframe(hor_8_raw_dataset_dict, 8)


/var/folders/88/mkn0vj0s1kl1179x22s6j2h80000gq/T/ipykernel_13903/3575878633.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(inplace=True)
/var/folders/88/mkn0vj0s1kl1179x22s6j2h80000gq/T/ipykernel_13903/3575878633.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns={'Inflation t+'+str(horizon): 'Actual Horizon '+str(horizon)}, inplace=True)
/var/folders/88/mkn0vj0s1kl1179x22s6j2h80000gq/T/ipykernel_13903/3575878633.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/s

In [17]:
def get_df_with_predictions_horizon(prediction_dic: dict, actual_dic:dict, horizon: int) -> dict:
    all_data_dict = {}
    for key in list(prediction_dic.keys()):
        if prediction_dic[key] == {}:
            continue
        predictions = prediction_dic[key][horizon].iloc[:,-1:].reset_index(drop = True)
        predictions = predictions.rename(columns = {f'Inflation t+{horizon}': f'Prediction Horizon {horizon}'})
        actuals = actual_dic[key]

        data = pd.concat([predictions, actuals], axis=1).loc[:,['Category','Date','Year',f'Prediction Horizon {horizon}', f'Actual Horizon {horizon}']]
        all_data_dict[key] = data
        
    return all_data_dict

In [18]:
all_data_test_dict_hor_2 = get_df_with_predictions_horizon(category_horizon_dict, test_dict_hor_2,2)
all_data_test_dict_hor_3 = get_df_with_predictions_horizon(category_horizon_dict, test_dict_hor_3,3)
all_data_test_dict_hor_4 = get_df_with_predictions_horizon(category_horizon_dict, test_dict_hor_4,4)
all_data_test_dict_hor_8 = get_df_with_predictions_horizon(category_horizon_dict, test_dict_hor_8,8)

In [19]:
all_data_test_dict_hor_2['All-items']

,Category,Date,Year,Prediction Horizon 2,Actual Horizon 2
0,All-items,2021-01-15,2021.0,0.372725,0.502694
1,All-items,2021-02-15,2021.0,0.168617,0.500180
2,All-items,2021-03-15,2021.0,0.328134,0.497690
3,All-items,2021-04-15,2021.0,0.083087,0.283286
4,All-items,2021-05-15,2021.0,0.597438,0.634475
5,All-items,2021-06-15,2021.0,0.154207,0.210600
6,All-items,2021-07-15,2021.0,0.352959,0.210158
7,All-items,2021-08-15,2021.0,-0.285091,0.697353
8,All-items,2021-09-15,2021.0,0.368506,0.208261
9,All-items,2021-10-15,2021.0,0.565292,-0.138793


In [20]:
def plot_results_horizon(all_data_dict, categories, horizon):
    #category_samples = random.sample(categories, 5)+['All items']
    category_samples = categories
    for category in category_samples:
        category_df = all_data_dict[category]
        y_pred = category_df[f'Prediction Horizon {horizon}'].values
        y_actual = category_df[f'Actual Horizon {horizon}'].values

        non_null_indices = ~np.isnan(y_pred) & ~np.isnan(y_actual)

        predicted_values_non_null = y_pred[non_null_indices]
        observed_values_non_null = y_actual[non_null_indices]

        mse = mean_squared_error(predicted_values_non_null, observed_values_non_null)
        print(f'Category is: {category}')
        print(f'RMSE is: {np.sqrt(mse)}')

        fig = px.line(category_df, x="Date", y=[f'Actual Horizon {horizon}', f'Prediction Horizon {horizon}'], title=f'{category} - Actual VS Prediction: horizon {horizon}')
        fig.show()



In [21]:
def total_corr_horizon(all_data_test_dict, horizon):
    corr_dict = {}
    for key in list(all_data_test_dict.keys()):
        df = all_data_test_dict[key]
        y_pred = df[f'Prediction Horizon {horizon}'].values
        y_actual = df[f'Actual Horizon {horizon}'].values

        non_null_indices = ~np.isnan(y_pred) & ~np.isnan(y_actual)

        predicted_values_non_null = y_pred[non_null_indices]
        observed_values_non_null = y_actual[non_null_indices]

        corr = stats.pearsonr(predicted_values_non_null,observed_values_non_null)[0]
        corr_dict[key] =  corr
    
    total_corr = sum(corr_dict.values())
    
    num_high_corr = 0
    for category in corr_dict:
        if corr_dict[category] >= 0.5:
            num_high_corr +=1
    
    print(f'Number of categories with High Correlation: {num_high_corr}')
    print(f'Total Corr Horizon {horizon}, {total_corr}')
    return total_corr

In [22]:
total_corr_horizon(all_data_test_dict_hor_2, 2)
total_corr_horizon(all_data_test_dict_hor_3, 3)
total_corr_horizon(all_data_test_dict_hor_4, 4)
total_corr_horizon(all_data_test_dict_hor_8, 8)

Number of categories with High Correlation: 8
Total Corr Horizon 2, -7.814214486155272
Number of categories with High Correlation: 6
Total Corr Horizon 3, -2.523508838355515
Number of categories with High Correlation: 6
Total Corr Horizon 4, -11.251767927868897
Number of categories with High Correlation: 6
Total Corr Horizon 8, -1.49019927056374


-1.49019927056374

In [23]:
cats = list(all_data_test_dict_hor_2.keys())
categories = random.sample(cats, 5)+['All-items']

In [24]:
#categories = list(all_data_test_dict_hor_2.keys())
plot_results_horizon(all_data_test_dict_hor_2, categories, 2)

Category is: Household appliances
RMSE is: 1.7005701415913377


Category is: Upholstered furniture
RMSE is: 2.7853001335736787


Category is: Digital computing equipment and devices
RMSE is: 2.7550627625898585


Category is: Other lessons, courses and education services
RMSE is: 1.109692619469533


Category is: Recreational services
RMSE is: 0.7985345038069596


Category is: All-items
RMSE is: 0.576685831593176


In [25]:
#categories = list(all_data_test_dict_hor_3.keys())
plot_results_horizon(all_data_test_dict_hor_3, categories, 3)

Category is: Household appliances
RMSE is: 1.9749626238949411


Category is: Upholstered furniture
RMSE is: 2.7211415412609674


Category is: Digital computing equipment and devices
RMSE is: 2.423981225253324


Category is: Other lessons, courses and education services
RMSE is: 1.351194389995814


Category is: Recreational services
RMSE is: 0.8374581285588789


Category is: All-items
RMSE is: 0.6017406471976616


In [26]:
#categories = list(all_data_test_dict_hor_4.keys())
plot_results_horizon(all_data_test_dict_hor_4, categories, 4)

Category is: Household appliances
RMSE is: 1.96145717110396


Category is: Upholstered furniture
RMSE is: 2.409114741793567


Category is: Digital computing equipment and devices
RMSE is: 2.377171187812355


Category is: Other lessons, courses and education services
RMSE is: 1.181128571960845


Category is: Recreational services
RMSE is: 0.8513872072056495


Category is: All-items
RMSE is: 0.6879228439895362


In [27]:
#categories = list(all_data_test_dict_hor_8.keys())
plot_results_horizon(all_data_test_dict_hor_8, categories, 8)

Category is: Household appliances
RMSE is: 1.8902140771201528


Category is: Upholstered furniture
RMSE is: 2.5547664498655993


Category is: Digital computing equipment and devices
RMSE is: 2.326208636090357


Category is: Other lessons, courses and education services
RMSE is: 1.232993640221623


Category is: Recreational services
RMSE is: 0.5833484127202448


Category is: All-items
RMSE is: 0.6759669282790712
